# TensorFlow自定义模型与训练算法

本notebook深入讲解TensorFlow/Keras的高级自定义功能，包括自定义损失函数、层、模型和训练循环。这些技术是从"调包侠"进阶到"算法工程师"的关键能力。

## 学习目标
1. 掌握自定义损失函数的实现方法
2. 学会自定义层（Layer）的创建
3. 理解自定义模型（Model）的构建
4. 实现自定义训练循环
5. 掌握模型的保存与加载

## 1. 环境设置与数据准备

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# 设置随机种子
RANDOM_SEED = 42
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")
print(f"Keras版本: {keras.__version__}")

In [ ]:
# 生成模拟回归数据集（模拟加州房价特征）
# 使用模拟数据避免网络依赖，同时保持数据结构一致

n_samples = 10000
n_features = 8  # 与加州房价数据集特征数一致

# 生成特征矩阵（已标准化）
X_full = np.random.randn(n_samples, n_features).astype(np.float32)

# 生成目标值（模拟房价，范围约0-5）
# 使用线性组合加噪声模拟真实关系
true_weights = np.random.randn(n_features).astype(np.float32)
y_full = X_full @ true_weights + np.random.randn(n_samples).astype(np.float32) * 0.5
y_full = (y_full - y_full.min()) / (y_full.max() - y_full.min()) * 5  # 归一化到0-5

# 划分数据集
train_size = int(0.64 * n_samples)  # 6400
valid_size = int(0.16 * n_samples)  # 1600
test_size = n_samples - train_size - valid_size  # 2000

X_train = X_full[:train_size]
y_train = y_full[:train_size]
X_valid = X_full[train_size:train_size + valid_size]
y_valid = y_full[train_size:train_size + valid_size]
X_test = X_full[train_size + valid_size:]
y_test = y_full[train_size + valid_size:]

print(f"训练集: {X_train.shape[0]}样本")
print(f"验证集: {X_valid.shape[0]}样本")
print(f"测试集: {X_test.shape[0]}样本")
print(f"特征数: {X_train.shape[1]}")

## 2. 自定义损失函数

### 2.1 函数式损失函数

最简单的自定义损失函数是一个接受`y_true`和`y_pred`两个参数的函数。

**Huber损失函数**是一种结合MSE和MAE优点的鲁棒损失函数：
- 当误差较小时（|error| < δ），使用MSE（对小误差敏感）
- 当误差较大时（|error| ≥ δ），使用MAE（对异常值鲁棒）

In [ ]:
def huber_fn(y_true, y_pred):
    """
    Huber损失函数实现
    
    参数:
        y_true: 真实标签
        y_pred: 预测值
        
    返回:
        损失值张量
        
    数学公式:
        L(y, f(x)) = 
            0.5 * (y - f(x))^2,           if |y - f(x)| < 1
            |y - f(x)| - 0.5,             otherwise
    """
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1.0
    squared_loss = tf.square(error) / 2.0
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# 测试Huber损失函数
y_true_test = tf.constant([1.0, 2.0, 3.0])
y_pred_test = tf.constant([1.5, 2.5, 5.0])  # 误差分别为0.5, 0.5, 2.0
loss = huber_fn(y_true_test, y_pred_test)
print(f"测试Huber损失: {loss.numpy()}")
print(f"  误差0.5 -> 使用MSE: 0.5^2/2 = 0.125")
print(f"  误差2.0 -> 使用MAE: |2.0| - 0.5 = 1.5")

In [ ]:
# 使用自定义损失函数训练模型

# 构建简单的回归模型
def build_regression_model(input_shape):
    """构建用于回归任务的MLP模型"""
    model = keras.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)  # 回归任务不需要激活函数
    ])
    return model

model = build_regression_model(input_shape=(X_train.shape[1],))
model.compile(loss=huber_fn, optimizer='adam')

# 使用简单参数进行快速测试
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_valid, y_valid),
    verbose=1
)

# 评估模型
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f"\n测试集损失: {test_loss:.4f}")

### 2.2 带参数的损失函数

如果损失函数需要可配置的参数（如Huber的阈值），可以使用工厂函数或类来实现。

In [ ]:
def create_huber(threshold=1.0):
    """
    创建指定阈值的Huber损失函数
    
    参数:
        threshold: 阈值δ，控制MSE和MAE的切换点
        
    返回:
        配置好的Huber损失函数
    """
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2.0
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2.0
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

# 使用不同阈值测试
huber_1 = create_huber(1.0)
huber_2 = create_huber(2.0)

error_values = tf.constant([0.5, 1.5, 3.0])
y_true = tf.constant([0.0, 0.0, 0.0])
y_pred = -error_values

print(f"误差值: {error_values.numpy()}")
print(f"阈值1.0的损失: {huber_1(y_true, y_pred).numpy()}")
print(f"阈值2.0的损失: {huber_2(y_true, y_pred).numpy()}")

### 2.3 继承keras.losses.Loss实现损失类

继承`keras.losses.Loss`类可以实现：
- 参数随模型一起保存
- 支持`get_config()`序列化
- 更好的代码组织

In [ ]:
class HuberLoss(keras.losses.Loss):
    """
    Huber损失函数类实现
    
    继承keras.losses.Loss可以使参数随模型保存
    """
    
    def __init__(self, threshold=1.0, **kwargs):
        """
        初始化Huber损失
        
        参数:
            threshold: 阈值δ
            **kwargs: 传递给父类的其他参数
        """
        super().__init__(**kwargs)
        self.threshold = threshold

    def call(self, y_true, y_pred):
        """计算损失值"""
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2.0
        linear_loss = self.threshold * tf.abs(error) - self.threshold ** 2 / 2.0
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        """返回配置字典，用于序列化"""
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# 测试损失类
huber_loss = HuberLoss(threshold=1.5)
print(f"损失类配置: {huber_loss.get_config()}")

# 使用损失类编译模型
model = build_regression_model(input_shape=(X_train.shape[1],))
model.compile(loss=HuberLoss(threshold=2.0), optimizer='adam')
print("模型编译成功，使用HuberLoss(threshold=2.0)")

## 3. 自定义激活函数、初始化器和正则化器

这些组件都可以通过简单的函数或类来自定义。

In [ ]:
# 自定义激活函数 - Softplus的手动实现
def my_softplus(z):
    """Softplus激活: log(exp(z) + 1)，是ReLU的平滑近似"""
    return tf.math.log(tf.exp(z) + 1.0)

# 自定义初始化器 - Glorot初始化
def my_glorot_initializer(shape, dtype=tf.float32):
    """Glorot/Xavier均匀初始化"""
    stddev = tf.sqrt(2.0 / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# 使用自定义组件创建层
# 注意：Keras 3.x中，正则化器和约束必须是相应基类的子类实例
# 使用内置组件演示功能
custom_layer = keras.layers.Dense(
    10,
    activation=my_softplus,
    kernel_initializer=my_glorot_initializer,
    kernel_regularizer=keras.regularizers.l1(0.001),
    kernel_constraint=keras.constraints.NonNeg()  # 非负约束
)

# 测试自定义层
test_input = tf.random.normal([2, 5])
output = custom_layer(test_input)
print(f"输入形状: {test_input.shape}")
print(f"输出形状: {output.shape}")
print(f"权重是否非负: {tf.reduce_all(custom_layer.kernel >= 0).numpy()}")

In [ ]:
# 可序列化的正则化器类

class MyL1Regularizer(keras.regularizers.Regularizer):
    """可序列化的L1正则化器"""
    
    def __init__(self, factor=0.001):
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(weights)) * self.factor

    def get_config(self):
        return {"factor": self.factor}

# 测试正则化器
reg = MyL1Regularizer(factor=0.01)
test_weights = tf.constant([[1.0, -2.0], [3.0, -4.0]])
reg_loss = reg(test_weights)
print(f"正则化损失: {reg_loss.numpy():.4f}")
print(f"配置: {reg.get_config()}")

## 4. 自定义评估指标

### 4.1 流式指标（Streaming Metric）

流式指标在整个训练过程中累积计算，而不是每个批次单独计算。

In [ ]:
class HuberMetric(keras.metrics.Metric):
    """
    流式Huber指标
    
    在整个epoch中累积计算平均Huber损失
    """
    
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        # 创建状态变量用于累积
        self.total = self.add_weight(name="total", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        """更新状态变量"""
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2.0
        linear_loss = self.threshold * tf.abs(error) - self.threshold ** 2 / 2.0
        loss = tf.where(is_small_error, squared_loss, linear_loss)
        
        self.total.assign_add(tf.reduce_sum(loss))
        self.count.assign_add(tf.cast(tf.size(loss), tf.float32))

    def result(self):
        """返回当前指标值"""
        return self.total / self.count

    def reset_state(self):
        """重置状态（每个epoch开始时调用）"""
        self.total.assign(0.0)
        self.count.assign(0.0)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# 测试自定义指标
metric = HuberMetric(threshold=1.0)

# 模拟多个批次的更新
metric.update_state(tf.constant([1.0, 2.0]), tf.constant([1.5, 2.2]))
metric.update_state(tf.constant([3.0, 4.0]), tf.constant([3.1, 5.5]))

print(f"累积Huber指标: {metric.result().numpy():.4f}")

# 重置
metric.reset_state()
print(f"重置后: {metric.result().numpy():.4f}")

## 5. 自定义层

### 5.1 无权重的自定义层

In [ ]:
class GaussianNoise(keras.layers.Layer):
    """
    高斯噪声层
    
    在训练时添加高斯噪声作为正则化手段，推理时不添加噪声
    """
    
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(
                shape=tf.shape(inputs),
                mean=0.0,
                stddev=self.stddev
            )
            return inputs + noise
        return inputs

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "stddev": self.stddev}

# 测试噪声层
noise_layer = GaussianNoise(stddev=0.1)
test_input = tf.constant([[1.0, 2.0, 3.0]])

print(f"原始输入: {test_input.numpy()}")
print(f"训练模式输出: {noise_layer(test_input, training=True).numpy()}")
print(f"推理模式输出: {noise_layer(test_input, training=False).numpy()}")

### 5.2 带权重的自定义层

In [ ]:
class MyDense(keras.layers.Layer):
    """
    自定义全连接层
    
    手动实现Dense层以理解权重管理机制
    """
    
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        """延迟创建权重，直到知道输入形状"""
        self.kernel = self.add_weight(
            name="kernel",
            shape=[input_shape[-1], self.units],
            initializer="glorot_uniform",
            trainable=True
        )
        self.bias = self.add_weight(
            name="bias",
            shape=[self.units],
            initializer="zeros",
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs):
        """前向传播"""
        z = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation is not None:
            return self.activation(z)
        return z

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "units": self.units,
            "activation": keras.activations.serialize(self.activation)
        }

# 测试自定义Dense层
my_dense = MyDense(5, activation='relu')
test_input = tf.random.normal([3, 4])
output = my_dense(test_input)

print(f"输入形状: {test_input.shape}")
print(f"输出形状: {output.shape}")
print(f"权重形状: {my_dense.kernel.shape}")
print(f"偏置形状: {my_dense.bias.shape}")

### 5.3 多输入/多输出层

In [ ]:
class MultiOutputLayer(keras.layers.Layer):
    """
    多输出计算层
    
    接收两个输入，返回它们的和、积、商
    """
    
    def call(self, inputs):
        x1, x2 = inputs
        return [
            x1 + x2,
            x1 * x2,
            x1 / (x2 + 1e-7)  # 添加小值避免除零
        ]

# 测试多输出层
multi_layer = MultiOutputLayer()
x1 = tf.constant([[1.0, 2.0]])
x2 = tf.constant([[3.0, 4.0]])
outputs = multi_layer([x1, x2])

print(f"输入x1: {x1.numpy()}")
print(f"输入x2: {x2.numpy()}")
print(f"和: {outputs[0].numpy()}")
print(f"积: {outputs[1].numpy()}")
print(f"商: {outputs[2].numpy()}")

## 6. 自定义模型

### 6.1 残差块和残差网络

In [ ]:
class ResidualBlock(keras.layers.Layer):
    """
    残差块
    
    实现跳跃连接：output = F(x) + x
    """
    
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.hidden = [
            keras.layers.Dense(
                n_neurons,
                activation="elu",
                kernel_initializer="he_normal"
            )
            for _ in range(n_layers)
        ]

    def call(self, inputs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        return inputs + z  # 残差连接

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "n_layers": self.n_layers,
            "n_neurons": self.n_neurons
        }

In [ ]:
class ResidualRegressor(keras.Model):
    """
    残差回归模型
    
    使用残差块构建的回归网络
    """
    
    def __init__(self, output_dim=1, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(
            30, activation="elu", kernel_initializer="he_normal"
        )
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        z = self.hidden1(inputs)
        z = self.block1(z)
        z = self.block2(z)
        return self.out(z)

# 测试残差模型
res_model = ResidualRegressor(output_dim=1)
test_input = tf.random.normal([5, 8])
output = res_model(test_input)

print(f"输入形状: {test_input.shape}")
print(f"输出形状: {output.shape}")
res_model.summary()

### 6.2 带辅助损失的模型

In [ ]:
class ReconstructingRegressor(keras.Model):
    """
    带重建损失的回归模型
    
    除了主要的回归任务外，还添加输入重建作为辅助任务，
    这有助于学习更好的特征表示
    """
    
    def __init__(self, output_dim=1, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [
            keras.layers.Dense(
                30, activation="selu", kernel_initializer="lecun_normal"
            )
            for _ in range(3)
        ]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruct = None  # 延迟创建

    def build(self, input_shape):
        n_inputs = input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(input_shape)

    def call(self, inputs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        
        # 计算重建损失并添加到模型
        reconstruction = self.reconstruct(z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)  # 添加辅助损失
        
        return self.out(z)

# 测试带辅助损失的模型
recon_model = ReconstructingRegressor()
recon_model.compile(loss='mse', optimizer='adam')

# 快速训练测试
history = recon_model.fit(
    X_train[:1000], y_train[:1000],
    epochs=3,
    batch_size=64,
    verbose=1
)
print(f"模型总损失包含主损失+辅助损失")

## 7. 自定义训练循环

当`model.fit()`无法满足需求时（如GAN训练、强化学习），需要自定义训练循环。

In [ ]:
def random_batch(X, y, batch_size):
    """随机采样一个批次"""
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

def print_status_bar(step, total, loss, metrics=None):
    """打印训练进度条"""
    metrics_str = " - ".join(
        [f"{m.name}: {m.result():.4f}" for m in (metrics or [])]
    )
    end = "" if step < total else "\n"
    print(f"\r{step}/{total} - loss: {loss:.4f} - {metrics_str}", end=end)

In [ ]:
# 创建模型和优化器
model = keras.Sequential([
    keras.layers.Dense(30, activation='elu', kernel_initializer='he_normal',
                      kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(1)
])

# 超参数
n_epochs = 3
batch_size = 32
n_steps = len(X_train) // batch_size

# 优化器和损失函数
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.MeanSquaredError()

# 指标
mean_loss = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

# 自定义训练循环
print("开始自定义训练循环...")
for epoch in range(n_epochs):
    print(f"\nEpoch {epoch + 1}/{n_epochs}")
    
    for step in range(n_steps):
        # 获取批次数据
        X_batch, y_batch = random_batch(X_train, y_train, batch_size)
        
        # 前向传播和梯度计算
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            # 加入正则化损失
            total_loss = main_loss + sum(model.losses)
        
        # 反向传播
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        # 更新指标
        mean_loss(total_loss)
        mae_metric(y_batch, y_pred)
        
        # 每100步打印一次
        if step % 100 == 0:
            print_status_bar(step, n_steps, mean_loss.result(), [mae_metric])
    
    # epoch结束打印
    print_status_bar(n_steps, n_steps, mean_loss.result(), [mae_metric])
    
    # 重置指标
    mean_loss.reset_state()
    mae_metric.reset_state()

print("\n训练完成!")

## 8. 保存和加载自定义组件

In [ ]:
import tempfile
import os

# 创建使用自定义组件的模型
model = keras.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=(8,)),
    keras.layers.Dense(1)
])
model.compile(loss=HuberLoss(threshold=1.5), optimizer='adam')

# 训练一个epoch
model.fit(X_train, y_train, epochs=1, verbose=0)

# 保存模型
with tempfile.TemporaryDirectory() as tmpdir:
    model_path = os.path.join(tmpdir, 'model_with_custom_loss.keras')
    model.save(model_path)
    print(f"模型已保存到: {model_path}")
    
    # 加载模型时需要提供自定义对象
    loaded_model = keras.models.load_model(
        model_path,
        custom_objects={"HuberLoss": HuberLoss}
    )
    print(f"模型加载成功!")
    
    # 验证
    original_pred = model.predict(X_test[:5], verbose=0)
    loaded_pred = loaded_model.predict(X_test[:5], verbose=0)
    print(f"预测结果一致: {np.allclose(original_pred, loaded_pred)}")

## 知识点总结

### 自定义组件速查表

| 组件 | 继承类 | 必须实现的方法 |
|-----|-------|---------------|
| 损失函数 | `keras.losses.Loss` | `call(y_true, y_pred)` |
| 评估指标 | `keras.metrics.Metric` | `update_state()`, `result()`, `reset_state()` |
| 层 | `keras.layers.Layer` | `build()`, `call()` |
| 模型 | `keras.Model` | `call()` |
| 正则化器 | `keras.regularizers.Regularizer` | `__call__(weights)` |

### 关键要点

1. **函数式组件**：简单场景使用普通函数即可
2. **类式组件**：需要保存配置时继承相应基类并实现`get_config()`
3. **延迟构建**：在`build()`方法中创建权重，支持动态输入形状
4. **自定义训练循环**：使用`tf.GradientTape`进行梯度计算
5. **保存/加载**：通过`custom_objects`参数传递自定义类